### 문제정의
- 버섯의 특징을 활용해서 식용인지 독성인지 분류해보자

### 문제데이터 수집

In [5]:
#  데이터 가져오기
import pandas as pd
data = pd.read_csv('./Data/mushroom.csv')
data.head()


,poisonous,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [7]:
# 데이터 갯수 확인
data.keys
# 행(8124), 열(특성 22개, 정답 1개)
data.shape # (행, 열)의 형태로 가져옴

(8124, 23)

### 데이터 전처리

In [8]:
# 결측치
data.info()
# 데이터에 결측치가 없다.

# 이상치
data.describe() # 수치 자료값만 통계로 확인시켜줌 
#describe() : 숫자인 컬럼에서만 의미가 있음
# 문자열로 이루어진 칼럼 
# unique : 컬럼을 구성하고 있는 문자의 가지수
# top : 어떤 데이터가 가장 많은지
# freq : top의 빈도수 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   poisonous                 8124 non-null   object
 1   cap-shape                 8124 non-null   object
 2   cap-surface               8124 non-null   object
 3   cap-color                 8124 non-null   object
 4   bruises                   8124 non-null   object
 5   odor                      8124 non-null   object
 6   gill-attachment           8124 non-null   object
 7   gill-spacing              8124 non-null   object
 8   gill-size                 8124 non-null   object
 9   gill-color                8124 non-null   object
 10  stalk-shape               8124 non-null   object
 11  stalk-root                8124 non-null   object
 12  stalk-surface-above-ring  8124 non-null   object
 13  stalk-surface-below-ring  8124 non-null   object
 14  stalk-color-above-ring  

,poisonous,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,...,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,2,6,4,10,2,9,2,2,2,12,...,4,9,9,1,4,3,5,9,6,7
top,e,x,y,n,f,n,f,c,b,b,...,s,w,w,p,w,o,p,w,v,d
freq,4208,3656,3244,2284,4748,3528,7914,6812,5612,1728,...,4936,4464,4384,8124,7924,7488,3968,2388,4040,3148


### 데이터 탐색적 분석
- 일단 생략 : 데이터에 수치형 데이터가 없기 때문에 시각화하기가 어려움

### 모델 선택 및 하이퍼파라미터 튜닝

#### 모델 불러오기

In [9]:
# DecsisionTree 모델 불러오기
from sklearn.tree import DecisionTreeClassifier
tree_model = DecisionTreeClassifier() #프럼에서 모델을 가져오면 클래스로 변환되기 때문에 소괄호 필요

#### 데이터 나누기

##### 문제와 정답데이터로 나누기

In [10]:
X = data.loc[:, 'cap-shape' : ] #loc ['가져오고싶은 문자행', '가져오고 싶은 문자 열'] 문자로 가져와도 슬라이싱 가능 
#문제

y =data.loc[:,'poisonous']
#정답

print(X)
print(y)

     cap-shape cap-surface cap-color bruises odor gill-attachment  \
0            x           s         n       t    p               f   
1            x           s         y       t    a               f   
2            b           s         w       t    l               f   
3            x           y         w       t    p               f   
4            x           s         g       f    n               f   
...        ...         ...       ...     ...  ...             ...   
8119         k           s         n       f    n               a   
8120         x           s         n       f    n               a   
8121         f           s         n       f    n               a   
8122         k           y         n       f    y               f   
8123         x           s         n       f    n               a   

     gill-spacing gill-size gill-color stalk-shape  ...  \
0               c         n          k           e  ...   
1               c         b          k           e  .

##### 문제를 숫자로 만들기 

In [11]:
X1 = X.copy()    # <==== 데이터를 변형하면서 원본은 납둬야 한다. copy()는 데이터를 복사해줌. : 다른 주소로 X, X1은 전혀 별개의 데이터, X1의 값이 바뀌어도 X에는 반영 안됨
X1

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,x,s,n,t,p,f,c,n,k,e,...,s,w,w,p,w,o,p,k,s,u
1,x,s,y,t,a,f,c,b,k,e,...,s,w,w,p,w,o,p,n,n,g
2,b,s,w,t,l,f,c,b,n,e,...,s,w,w,p,w,o,p,n,n,m
3,x,y,w,t,p,f,c,n,n,e,...,s,w,w,p,w,o,p,k,s,u
4,x,s,g,f,n,f,w,b,k,t,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,k,s,n,f,n,a,c,b,y,e,...,s,o,o,p,o,o,p,b,c,l
8120,x,s,n,f,n,a,c,b,y,e,...,s,o,o,p,n,o,p,b,v,l
8121,f,s,n,f,n,a,c,b,n,e,...,s,o,o,p,o,o,p,b,c,l
8122,k,y,n,f,y,f,c,n,b,t,...,k,w,w,p,w,o,e,w,v,l


###### Label 인코딩
- 하나의 값에 하나의 숫자를 대입
- 많이 사용되는 방법은 아님
  - 이유는? 값을 변경함에 있어서 사용의 개입이 들어감.
  - 원래 문자가 가지고 있던 의미는 사라지고 사용자가 임의대로 값을 바꿔버림.

In [12]:
X1['cap-shape'].unique()
# 총 6개의 값을 값이 가지고 있는 순서대로 번호(1~6)을 문자값에 매칭되는 수치형 데이터로 변형
 # 'x' : 1 , 'b ' : 2
 # => 파이썬의 인덱스 갠며을 적용해서 0~5번으로 변경

array(['x', 'b', 's', 'f', 'k', 'c'], dtype=object)

In [13]:
# Label 인코딩 : map함수 
# map()을 적용하기 위해 딕셔너리형태로 값을 저장
trans = {'x' : 0, 'b' : 1, 's' : 2, 'f':3,'k':4 , 'c' : 5}

# key값을 value 값으로 변경
X1['cap-shape'] = X1['cap-shape'].map(trans)

#map() 함수를 쓸 떄 주의할 점 : 가지고 있는 대상을 전부 바꿔줘야함 

In [15]:
X1['cap-shape'].value_counts() # 값이 바뀌는 것을 decribe() 수치와 비교 호가인 

0    3656
3    3152
4     828
1     452
2      32
5       4
Name: cap-shape, dtype: int64

###### One-hot 인코딩
- 컬럼이 가지고 있는 데이터만큼의 컬럼을 또 생성
- 0과 1로 데이터를 표시
- 데이터가 늘어나는 효과를 얻음
- Label인코딩보다 더욱 많이 사용하게됨.

- Label vs One_hot
    - Label : 기존에 가지고 있던 데이터의 의미가 사라짐
    - One-hot : 장점1. 기존에 가지고 있는 데이터의 의미 유지 
                    2. 데이터가 늘어나는 효과 - 가지수가 많아짐
![image.png]

In [20]:
# One-hot 인코딩 진행하는 함수 : pd.get_dummies(대상)
X2 = X.copy()
X_onehot = pd.get_dummies(X2)
X_onehot.head() #X_onehot에는 숫자형 데이터가 담겨있음. 원래 문자로 담겨 있는 컬럼은 cap-shape_'b' 처럼 뒤에 문자로 나누어져 있음.

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,0,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


#### 훈련과 평가 데이터로 나누기(train/test)

In [22]:
#문제와 정답데이터를 훈련과 평가용으로 다시 나눠주기 -> 함수이름
# 문제 : X_onehot
# 정답 : y
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_onehot, y, test_size=0.3)

In [23]:
X_train.shape

(5686, 117)

### 학습

In [24]:
tree_model.fit(X_train, y_train)

DecisionTreeClassifier()

### 평가

In [27]:
# 보통 1.0 => 과대적합
tree_model.score(X_train, y_train)

1.0

In [29]:
tree_model.score(X_test, y_test)

# 학습용, 평가용 모두 1.0
# 1.뭔가 잘못됐다 : 99%
# 2.지나치게 학습이 잘 됐다 : 1% --> 데이터가 구분하기 좋은 데이터로 구성되어있어서 학습이 잘 됬다.

1.0